## Download Dataset

In [ ]:
from graphviper.utils.data import download

download(file="Antennae_North.cal.lsrk.vis.zarr")

## Setup Dask Cluster

In [ ]:
# import dask
# dask.config.set(scheduler="synchronous")

from graphviper.dask.client import local_client
viper_client = local_client(cores=4, memory_limit="4GB")
# log_params = {'log_to_term':True,'log_level':'DEBUG'}
# viper_client = local_client(cores=16, memory_limit="10GB",log_params=log_params,worker_log_params=log_params)
viper_client

## Inspect Processing Set

In [ ]:
from xradio.vis.read_processing_set import read_processing_set

intents = ["OBSERVE_TARGET#ON_SOURCE"]
ps_name = "Antennae_North.cal.lsrk.vis.zarr"
ps = read_processing_set(ps_name, intents=intents)
ps.summary()

In [ ]:
ps.get(0)

## Run Cube Imaging Niter 0

In [ ]:
ps_name = "Antennae_North.cal.lsrk.vis.zarr"
image_name = "Antennae_North_Cube.img.zarr"

import numpy as np
import os

grid_params = {}
grid_params["chan_mode"] = "cube"
grid_params["image_size"] = [500, 500]
grid_params["cell_size"] = np.array([-0.13, 0.13]) * np.pi / (180 * 3600)
grid_params["fft_padding"] = 1.0
grid_params["phase_direction"] = 12

data_variables = ["sky", "point_spread_function", "primary_beam"]

os.system("rm -rf " + image_name)
n_chunks = 60
from astroviper.imaging.cube_imaging_niter0 import cube_imaging_niter0

cube_imaging_niter0(ps_name, image_name, grid_params, n_chunks, data_variables)

## Inspect Image

In [ ]:
import xarray as xr

img_xds = xr.open_zarr("Antennae_North_Cube.img.zarr")
img_xds

In [ ]:
import matplotlib.pyplot as plt

%matplotlib widget
plt.figure()
img_xds.POINT_SPREAD_FUNCTION.isel(polarization=1, frequency=82).plot(cmap='viridis',vmin=0.0)
plt.figure()
img_xds.PRIMARY_BEAM.isel(polarization=0, frequency=0).plot()
plt.figure()
img_xds.SKY.max(dim="frequency").isel(polarization=0).plot(cmap='viridis',vmin=0.0)

In [ ]:
#ipympl, pip install ipywidgets